# Exercise: Recurrent neural network language models

In this lab you will implement the recurrent neural network model from Lecture&nbsp;2.5. You will evaluate these models by computing their perplexity on a benchmark dataset.

In [ ]:
import torch

For this lab, you should use the GPU if you have one:

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Data

The data for this lab is [WikiText](https://blog.einstein.ai/the-wikitext-long-term-dependency-language-modeling-dataset/), a collection of more than 100 million tokens extracted from the set of &lsquo;Good&rsquo; and &lsquo;Featured&rsquo; articles on Wikipedia. We will use the small version of the dataset, which contains slightly more than 2.5 million tokens.

The next cell contains code for an object that will act as a container for the &lsquo;training&rsquo; and the &lsquo;validation&rsquo; section of the data. We fill this container by reading the corresponding text files. The only processing that we do is to whitespace-tokenize, and to replace each newline character with a special token `<eos>` (end-of-sentence).

In [ ]:
class WikiText(object):
    
    def __init__(self):
        self.vocab = {}
        self.train = self.read_data('wiki.train.tokens')
        self.valid = self.read_data('wiki.valid.tokens')
    
    def read_data(self, path):
        ids = []
        with open(path) as source:
            for line in source:
                for token in line.split() + ['<eos>']:
                    if token not in self.vocab:
                        self.vocab[token] = len(self.vocab)
                    ids.append(self.vocab[token])
        return ids

The cell below loads the data and prints the total number of tokens and the size of the vocabulary.

In [ ]:
wikitext = WikiText()

print('Tokens in train:', len(wikitext.train))
print('Tokens in valid:', len(wikitext.valid))
print('Vocabulary size:', len(wikitext.vocab))

## Recurrent neural network language model

In this section you will implement the recurrent neural network language model that was presented in Lecture&nbsp;2.5. Recall that an input to the network is a vector of word ids. Each integer is mapped to an embedding vector. The sequence of embedded vectors is then fed into an unrolled LSTM. At each position $i$ in the sequence, the hidden state of the LSTM at that position is sent through a linear transformation into a final softmax layer, from which we read off the index of the word at position $i+1$. In theory, the input vector could represent the complete training data or at least a complete sentence; for practical reasons, however, we will truncate the input to some fixed value *bptt_len*, the **backpropagation-through-time horizon**.

### Problem 1: Vectorize the data

Your first task is to transform the data in the WikiText container into a vectorized form that can be fed to the model.

In [ ]:
def vectorize_rnn(wikitext_data, bptt_len):
    # TODO: Replace the next line with your own code
    return None

In [ ]:
def vectorize_rnn(wikitext_data, bptt_len):
    xs, ys = [], []
    for i in range(0, len(wikitext_data) - bptt_len, bptt_len):
        xs.append(wikitext_data[i:i+bptt_len])
        ys.append(wikitext_data[i+1:i+bptt_len+1])
    return torch.LongTensor(xs), torch.LongTensor(ys)

Your function should meet the following specification:

**vectorize_rnn** (*wikitext_data*, *bptt_len*)

> Transforms a list of token indexes into a pair of tensors $\mathbf{X}$, $\mathbf{Y}$ that can be used to train the recurrent neural language model. The rows of both tensors represent contiguous subsequences of token indexes of length *bptt_len*. Compared to the sequences in $\mathbf{X}$, the corresponding sequences in $\mathbf{Y}$ are shifted one position to the right. More precisely, if the $i$th row of $\mathbf{X}$ is the sequence that starts at token position $j$, then the same row of $\mathbf{Y}$ is the sequence that starts at position $j+1$.

#### 🤞 Test your code

Test your implementation by running the following code:

In [ ]:
valid_x, valid_y = vectorize_rnn(wikitext.valid, 32)

print(valid_x.size())

### Problem 2: Implement the model

Your next task is to implement the recurrent neural network model based on the graphical specification given in the lecture.

In [ ]:
import torch.nn as nn

class RNNModel(nn.Module):
    
    def __init__(self, n_words, embedding_dim=50, hidden_dim=50):
        super().__init__()
        # TODO: Add your own code

    def forward(self, x):
        # TODO: Replace the next line with your own code
        raise NotImplemented

In [ ]:
import torch.nn as nn

class RNNModel(nn.Module):
    
    def __init__(self, n_words, embedding_dim=50, hidden_dim=50):
        super().__init__()
        self.n_words = n_words
        self.embedding = nn.Embedding(n_words, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, n_words)

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.rnn(embedded)
        return self.linear(output)

Your implementation should follow this specification:

**__init__** (*self*, *n_words*, *embedding_dim* = 50, *hidden_dim* = 50)

> Creates a new recurrent neural network language model. The argument *n_words* is the number of words in the vocabulary. The arguments *embedding_dim* and *hidden_dim* specify the dimensionalities of the embedding layer and the LSTM hidden layer, respectively; their default value is 50.

**forward** (*self*, *x*)

> Computes the network output on an input batch *x*. The shape of *x* is $(B, H)$, where $B$ is the batch size and $H$ is the length of each input sequence. The shape of the output tensor is $(B, H, V)$, where $V$ is the size of the vocabulary.

#### 🤞 Test your code

Test your code by instantiating the model and feeding it a batch of examples from the training data.

### Problem 2.3: Train the model

The training loop for the recurrent neural network model is essentially identical to the loop that you wrote for the feed-forward model. The only thing to note is that the cross-entropy loss function expects its input to be a two-dimensional tensor; you will therefore have to re-shape the output tensor from the LSTM as well as the gold-standard output tensor in a suitable way. The most efficient way to do so is to use the [`view()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.view) method.

In [ ]:
def train_rnn(n_epochs=1, batch_size=100, bptt_len=32):
    # TODO: Replace the next line with your own code
    return None

In [ ]:
import numpy as np
import torch.nn.functional as F
import torch.optim as optim

def train_rnn(n_epochs=1, batch_size=100, bptt_len=32, lr=1e-2):
    # Vectorize the data
    train_x, train_y = vectorize_rnn(wikitext.train, bptt_len)
    valid_x, valid_y = vectorize_rnn(wikitext.valid, bptt_len)

    # Initialize the model
    model = RNNModel(len(wikitext.vocab), embedding_dim=50, hidden_dim=50).to(device)

    # Initialize the optimizer
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Minimal validation perplexity seen so far
    min_ppl = float('inf')

    for t in range(n_epochs):
        # Training
        model.train()
        with tqdm(total=len(train_x)) as pbar:
            pbar.set_description(f'Epoch {t+1}')
            for bx, by in batchify(train_x, train_y, batch_size):
                optimizer.zero_grad()
                output = model.forward(bx).view(-1, len(wikitext.vocab))
                loss = F.cross_entropy(output, by.view(-1))
                loss.backward()
                optimizer.step()
                pbar.set_postfix(loss=loss.item(), ppl=np.exp(loss.item()))
                pbar.update(len(bx))

        # Evaluation
        model.eval()
        with torch.no_grad():
            losses = []
            for bx, by in batchify(valid_x, valid_y, batch_size):
                output = model.forward(bx).view(-1, len(wikitext.vocab))
                losses.append(F.cross_entropy(output, by.view(-1)).item())
            ppl = np.exp(sum(losses) / len(losses))
            print(f'Perplexity after epoch {t+1}: {ppl}', flush=True)

            # Terminate the training if the validation perplexity has not improved
            if ppl <= min_ppl - 2:
                min_ppl = ppl
            else:
                break

    return model

Here is the specification of the training function:

**train_rnn** (*n_epochs* = 1, *batch_size* = 100, *bptt_len* = 32, *lr* = 0.01)

> Trains a recurrent neural network language model on the WikiText data using minibatch gradient descent and returns it. The parameters *n_epochs* and *batch_size* specify the number of training epochs and the minibatch size, respectively. The parameter *bptt_len* specifies the length of the backpropagation-through-time horizon, that is, the length of the input and output sequences. Training uses the cross-entropy loss function and the [Adam optimizer](https://pytorch.org/docs/stable/optim.html#torch.optim.Adam) with learning rate *lr*. After each epoch, prints the perplexity of the model on the validation data.

Evaluate your model by running the following code cell:

In [ ]:
model_rnn = train_rnn(n_epochs=1)

**⚠️ You should achieve a validation perplexity of at most 310.**

## Problem 3: Parameter initialization (reflection)

Since the error surfaces that gradient search explores when training neural networks can be very complex, it is important to choose &lsquo;good&rsquo; initial values for the parameters. In PyTorch, the weights of the embedding layer are initialized by sampling from the standard normal distribution $\mathcal{N}(0, 1)$. Test how changing the standard deviation and/or the distribution affects the perplexity of your language model.